In [1]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader(input_files= ["NotariesAct.pdf"]).load_data()

In [2]:
print(documents)

[Document(id_='ccbe7993-d337-4df7-a884-6d141578eabb', embedding=None, metadata={'page_label': 'i', 'file_name': 'NotariesAct.pdf', 'file_path': 'NotariesAct.pdf', 'file_type': 'application/pdf', 'file_size': 335672, 'creation_date': '2024-09-23', 'last_modified_date': '2024-09-23'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Laws.Africa Legislation Commons\nMauritius\nNotaries Act\nAct 7 of 2008\nLegislation as at 30 June 2017\nFRBR URI: /akn/mu/act/2008/7/eng@2017-06-30\nThere may have been updates since this ﬁle was created.\nPDF created on 21 February 2024 at 15:44.\nCollection last checked for updates: 30 June 2017.\nCheck for updates\nAbout this collection\nThe legislation in this collection has been reproduced as it was originally printed in t

In [3]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model = HuggingFaceEmbedding()

c:\Users\shaan\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

In [5]:
db = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db.get_or_create_collection("acts_collection")

In [6]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [7]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

In [8]:
db2 = chromadb.PersistentClient(path="./chroma_db")
chroma_collection = db2.get_or_create_collection("acts_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)

In [9]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama3.1", request_timeout=420.0)

In [10]:
query_engine = index.as_query_engine(llm=llm)

In [12]:
response = query_engine.query("Where should the office of a notary be situated ?")
print(response)

The office of a notary shall be situated— in the island of Mauritius, in the District of Port Louis; or in Rodrigues, in Port Mathurin.
